In [ ]:
import os
import json
import time
from datetime import datetime, timedelta
"""
This part of library are Python inbuild.
- os Library for setup path for server
- json Libray for converting API data to Python object
- datetime and 'timedelta' for converting date object to Python object and 'datetime' is for geting current datetime
"""


import numpy as np
import pandas as pd
import requests
from dotenv import load_dotenv
"""
This part of library are 3rd party packages
- Numpy and Pandas for data manipulation
- requests is for fetching html or API data
  POST and GET are two common requests method for fetching data. POST request is for sending request with data to server,
  GET is common request we send to websites
- dotenv is for reading environment variable inside Python script like user name, password or other private data
"""



from whizzbox import (
    amazon_sites,
    config,
    custom_errors,
    db_connector,
    gsheet_connector as gc,
    s3_connector as s3c,
    toolkit,
)

"""
Wizzbox is private Python pacakge, that build by company it self for personal use
"""

In [ ]:
PROJECT_NAME = 'whiz-amz-defaulters'
SAMPLE = True  # if it's a sample, no. of sites are limited
SEND_FAIL_EMAIL = True
UPDATE_GSHEET = False
UPLOAD_TO_S3 = True

"""
all capital latter is writing code best practive in Python,
that indecate should not remove or change this variables.
"""

#### **Function 1 code** 

In [ ]:
# noinspection SpellCheckingInspection  (Function 1)
def get_service_area_id():
    """
    Retrieves service area IDs from an API endpoint.

    Returns:
        pd.DataFrame: DataFrame containing service area information.
    """

    payload = ""
    headers = {"cookie": "session-id=260-6428466-7223316; session-id-time=2082787201l; "
                         "session-token=GbElPtRlx37WtbcOToTC3cRl2schxCQbGfw9PermyrbyTyonOdmMyDvN9xAn3Y1u"
                         "urDFHnNpykoR0H9l8R%2Bw0pqKOD14Zwtk4QClUVQpQsIItI6ETeBDN0M5SpFCUku0BGvpR2xSOFJN"
                         "nMUMDfYiflLntxjcLhKYtat1HekaJ2LZEwTlJ3dL6%2Fs%2B9U37Zu8JIQq%2FK7qPG5HFr4SwvqwIoQ"
                         "pyvIMoFI7ZbtIDa1gNSij8nXYZmqmACRQ0FUVbIFHYlcHjlwmkXzg%3D"}

    url = "https://logistics.amazon.in/flex/api/getOperationalRegions"
    response = requests.get(url, data=payload, headers=headers)

    data = json.loads(response.text)
    req_data = data
    api_df_raw = pd.DataFrame(req_data)
    
    """
    Payload, Header, and URl is sending request materials.
    Header conferm, we get write data based on our requirements and URL is parrent path.
    Request library under the hood make this into url raw string, that server can understand.
    
    We converted APIs json response to Python json object help of json.loads()
    We converted json object to pandas data frame 'api_df_raw'
    """
    
    

    # Explode the 'basicServiceAreas' column into multiple rows
    api_df = api_df_raw.explode('basicServiceAreas')

    # Split the exploded column into separate columns
    api_df_inter = pd.concat(
        [
            api_df.drop(['basicServiceAreas'], axis=1),
            api_df['basicServiceAreas'].apply(pd.Series)
        ],
        axis=1
    )

    # Concatenate the necessary columns for the final DataFrame
    df_final = pd.concat(
        [
            api_df_inter.drop(['pickUpLocationAddress', 'pickUpLocation'], axis=1),
            api_df_inter['pickUpLocationAddress'].apply(pd.Series),
            api_df_inter['pickUpLocation'].apply(pd.Series)
        ],
        axis=1
    )

    # Select the desired columns and rename them
    final_df = df_final[['defaultStationCode', 'serviceAreaID', 'regionID', 'regionName',
                         'state', 'postalCode', 'longitude', 'latitude', 'active']]
    final_df.columns = ['site_code', 'service_area_id', 'region_id', 'region_name',
                        'state', 'postal_code', 'longitude', 'latitude', 'active']
    
    """
    This part doing data manupulation task based on business requirements, and return a final clean data frame
    """

    return final_df


In [ ]:
# Function 1 code break down
import json
import requests


url = "https://logistics.amazon.in/flex/api/getOperationalRegions"
payload = ""
headers = {"cookie": "session-id=260-6428466-7223316; session-id-time=2082787201l; "
                     "session-token=GbElPtRlx37WtbcOToTC3cRl2schxCQbGfw9PermyrbyTyonOdmMyDvN9xAn3Y1u"
                     "urDFHnNpykoR0H9l8R%2Bw0pqKOD14Zwtk4QClUVQpQsIItI6ETeBDN0M5SpFCUku0BGvpR2xSOFJN"
                     "nMUMDfYiflLntxjcLhKYtat1HekaJ2LZEwTlJ3dL6%2Fs%2B9U37Zu8JIQq%2FK7qPG5HFr4SwvqwIoQ"
                     "pyvIMoFI7ZbtIDa1gNSij8nXYZmqmACRQ0FUVbIFHYlcHjlwmkXzg%3D"}

response = requests.get(url, data=payload, headers=headers)

data = json.loads(response.text)
req_data = data

"""
# Display json data
file_path = "f1_get_service_area_id.json"
with open(file_path, 'w') as file:
    json.dump(data, file)
    print(f"===== Save json data on f1_get_service_area_id.json file, Jupyerlab have data limitaion ======")
""" 

api_df_raw = pd.DataFrame(req_data)
api_df = api_df_raw.explode('basicServiceAreas')
print(f"===== api_df data frame raws and columns: {api_df.shape}")


"""
This code is restructure the data base and return a clean data frame
"""
# Split the exploded column into separate columns
api_df_inter = pd.concat(
    [
        api_df.drop(['basicServiceAreas'], axis=1),
        api_df['basicServiceAreas'].apply(pd.Series)
    ],
    axis=1
)

# Concatenate the necessary columns for the final DataFrame
df_final = pd.concat(
    [
        api_df_inter.drop(['pickUpLocationAddress', 'pickUpLocation'], axis=1),
        api_df_inter['pickUpLocationAddress'].apply(pd.Series),
        api_df_inter['pickUpLocation'].apply(pd.Series)
    ],
    axis=1
)

# Select the desired columns and rename them
final_df = df_final[['defaultStationCode', 'serviceAreaID', 'regionID', 'regionName',
                     'state', 'postalCode', 'longitude', 'latitude', 'active']]
final_df.columns = ['site_code', 'service_area_id', 'region_id', 'region_name',
                    'state', 'postal_code', 'longitude', 'latitude', 'active']

print(f"===== final_df data frame raws and columns{final_df.shape} ====")

#### **Function 2 code** 

In [ ]:
def get_drivers_data(yyyy_mm_dd, service_area_id):
    """
    Retrieves driver data for a specific date and service area.

    Args:
        yyyy_mm_dd (str): The date in the format "YYYY-MM-DD".
        service_area_id (str): The service area ID.

    Returns:
        pd.DataFrame: DataFrame containing driver data.
    """

    querystring = {"localDate": f"{yyyy_mm_dd}", "serviceAreaId": f"{service_area_id}"}

    payload = ""
    headers = {
        "cookie": 'session-id=258-5953668-1245245; ubid-acbin=261-0863908-3910552; '
                  'lc-acbin=en_IN; x-acbin="57E9eKz@7A@To6Vddbaqer5jgdCIqlKfP2T8kM6wKCICy?gDv7vle?RKDpKPgjZK";'
                  ' at-acbin=Atza|IwEBIJk3W45PoKDQRfBKht6R7k4X7GddLPM54_esWjs8dHrH6815nPGyW5DMuqexf1w0bDDz10cOG'
                  'JJEgdOwPR3Us-wmX2ChldzceR7pz7ciQRrZdKYRDV5A-L0kRQRtlbXvJvefFPPYLQUohVZ59kgsrTlSnAMO2MUVSxI8q'
                  'sBOXW2fPqGjf3o3IIx_f3T8ASPkGXUQcS2C9yTpnncFHoyCgAcCfs0dyQBBJzFYedq3XUQLVg; sess-at-acbin'
                  '="ckW6tTbMs1yigrh3KsKaTVeibCPC2YHIFdC5PnL4F+Q="; x-amz-log-portal-locale=en-IN; session'
                  '-id-time=2082787201l; session-token=F8JjOaEew4Id8WCibAXIjaVT8YkIKwPtKpnJQf0jBIrqTbaLb4Y2Kv47'
                  'nJXwvdCTKygovIhu+XF4jOcoVSZ7ImRYrmDOYEUTdlQ6uYK2r9jmnmHdDBLkl/HGcgDFFKrrJidRt61RI4MJ1PGgzhW'
                  'OIWRMV7OAW+QVoHlg45HkGmhxM9AGo2+x2Ef52i5bwyjCBkSruiVfXAjKD+lMC/jz+1xLh2P9+ZIGbPtnFScrRdjrq'
                  'FJEehq9CQ',
        "authority": "logistics.amazon.in",
        "accept": "application/json, text/plain, */*",
        "accept-language": "en-GB,en-US;q=0.9,en;q=0.8",
        "referer": f"https://logistics.amazon.in/operations/execution/itineraries?selectedDay={yyyy_mm_dd}"
                   f"&serviceAreaId={service_area_id}",
        "sec-ch-ua": '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"Windows"',
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-origin",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/109.0.0.0 Safari/537.36"
    }

    
    

    url = "https://logistics.amazon.in/operations/execution/api/summaries"
    response = requests.get(url, data=payload, headers=headers, params=querystring)
    
    data = json.loads(response.text)
    driver_df = pd.DataFrame(data['transporters'])
    
    """
    This part same as function 1,
    that sending request with querystring, payload, and headers.
    After sending request it convert json response to pandas data frame
    """

    
    if not driver_df.empty:
        # Retrieve unique driver IDs
        driver_ids = list(driver_df['transporterId'].unique())

        itinerary_df = pd.DataFrame(data['itinerarySummaries'])
        packages_df = pd.DataFrame(data['transporterPackageSummaries']) if data['transporterPackageSummaries'] \
            else pd.DataFrame()

        # Get drivers' location information
        location_df = get_drivers_location(yyyy_mm_dd, service_area_id, driver_ids)

        # Merge driver, itinerary, and location data
        merged_df = pd.merge(itinerary_df, pd.merge(driver_df, location_df, on='transporterId', how='left'),
                             on='transporterId', how='left')
        merged_df['driver_name'] = merged_df['firstName'].str.title() + ' ' + merged_df['lastName'].str.title()

        if not packages_df.empty:
            # Merge package data if available
            merged_df_final = pd.merge(merged_df, packages_df[['transporterId', 'packageStatus']],
                                       on='transporterId', how='left')
        else:
            merged_df_final = merged_df.copy()

        df_final = merged_df_final.copy()
    else:
        df_final = pd.DataFrame()
        
    
    """
    This part check if data frame is contain data or not, if dat is there then it doing data manipulation task.
    After that it return pandas data frame for future process.
    """

    return df_final


#### **Function 3 code** 

In [ ]:
def get_drivers_location(yyyy_mm_dd, service_area_id, transporter_ids: list):
    """
    Retrieves the location data of drivers for a specific date, service area, and transporter IDs.

    Args:
        yyyy_mm_dd (str): The date in the format "YYYY-MM-DD".
        service_area_id (str): The service area ID.
        transporter_ids (list): List of transporter IDs.

    Returns:
        pd.DataFrame: DataFrame containing the drivers' location data.
    """

    # Prepare the payload with transporter IDs
    payload = {
        "transporterIds": transporter_ids
    }

    # Prepare the headers for the request
    headers = {
        "authority": "logistics.amazon.in",
        "accept": "application/json, text/plain, */*",
        "accept-language": "en-GB,en-US;q=0.9,en;q=0.8",
        "content-type": "application/json;charset=UTF-8",
        "cookie": 'session-id=258-5953668-1245245; ubid-acbin=261-0863908-3910552; '
                  'lc-acbin=en_IN; x-acbin="57E9eKz@7A@To6Vddbaqer5jgdCIqlKfP2T8kM6wKCICy?gDv7vle?RKDpKPgjZK";'
                  ' at-acbin=Atza|IwEBIJk3W45PoKDQRfBKht6R7k4X7GddLPM54_esWjs8dHrH6815nPGyW5DMuqexf1w0bDDz10cOG'
                  'JJEgdOwPR3Us-wmX2ChldzceR7pz7ciQRrZdKYRDV5A-L0kRQRtlbXvJvefFPPYLQUohVZ59kgsrTlSnAMO2MUVSxI8q'
                  'sBOXW2fPqGjf3o3IIx_f3T8ASPkGXUQcS2C9yTpnncFHoyCgAcCfs0dyQBBJzFYedq3XUQLVg; sess-at-acbin'
                  '="ckW6tTbMs1yigrh3KsKaTVeibCPC2YHIFdC5PnL4F+Q="; x-amz-log-portal-locale=en-IN; session'
                  '-id-time=2082787201l; session-token=F8JjOaEew4Id8WCibAXIjaVT8YkIKwPtKpnJQf0jBIrqTbaLb4Y2Kv47'
                  'nJXwvdCTKygovIhu+XF4jOcoVSZ7ImRYrmDOYEUTdlQ6uYK2r9jmnmHdDBLkl/HGcgDFFKrrJidRt61RI4MJ1PGgzhW'
                  'OIWRMV7OAW+QVoHlg45HkGmhxM9AGo2+x2Ef52i5bwyjCBkSruiVfXAjKD+lMC/jz+1xLh2P9+ZIGbPtnFScrRdjrq'
                  'FJEehq9CQ', "origin": "https://logistics.amazon.in",
        "referer": f"https://logistics.amazon.in/operations/execution/itineraries?selectedDay={yyyy_mm_dd}"
                   f"&serviceAreaId={service_area_id}",
        "sec-ch-ua": '"Chromium";v="110", "Not A(Brand";v="24", "Google Chrome";v="110"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": "'Windows'",
        "sec-fetch-dest": "empty",
        "sec-fetch-mode": "cors",
        "sec-fetch-site": "same-origin",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/110.0.0.0 Safari/537.36"
    }

    # Send a POST request to retrieve the drivers' location data
    url = "https://logistics.amazon.in/operations/execution/api/transporters/locationUpdate"
    response = requests.request("POST", url, json=payload, headers=headers)
    data = json.loads(response.text)
    """
    This part is same as function 1 and function 2 like sending request with querystring, payload, and headers.
    And also make a pandas dataframe.
    """
    
    
    # Convert the retrieved data into a DataFrame
    df = pd.DataFrame(data['transportersLocation'])
    """
    This part convert json object to pandas data frame only 'transportersLocation', that will found on API response
    """
    
    return df


#### **Function 4 code** 

In [ ]:
def manipulate_the_data(input_df):
    """
    Manipulates the input DataFrame to derive additional columns and rename existing columns.

    Args:
        input_df (pandas.DataFrame): The input DataFrame to manipulate.

    Returns:
        pandas.DataFrame: The manipulated DataFrame.
    """
    # Create a copy of the input DataFrame
    df_raw = input_df.copy()

    # Convert the DataFrame to object type
    df_inter = df_raw.astype(object)

    # Fill missing values with 'No Data'
    df_inter.fillna('No Data', inplace=True)

    # Concatenate columns and expand nested columns
    df = pd.concat([df_inter.drop(['stopProgress', 'packageStatus'], axis=1),
                    df_inter['stopProgress'].apply(pd.Series),
                    df_inter['packageStatus'].apply(pd.Series)], axis=1)

    # Convert timestamp columns to datetime format
    df['amazon_login_time'] = df['itineraryStartTime'].apply(
        lambda x: datetime.fromtimestamp(int(str(x)[:10])) if x != 'No Data' else None
    )
    df['location_updated_time'] = df['epochTimestamp'].apply(
        lambda x: datetime.fromtimestamp(int(str(x)[:10])) if x != 'No Data' else None
    )
    df['lastDriverEventTime'] = df['lastDriverEventTime'].apply(
        lambda x: datetime.fromtimestamp(int(str(x)[:10])) if x != 'No Data' else None
    )

    # Calculate inactive_since column
    df['inactive_since'] = np.where(df['location_updated_time'] >= df['lastDriverEventTime'],
                                    df['location_updated_time'],
                                    df['lastDriverEventTime'])
    df['inactive_since'].fillna(df['location_updated_time'], inplace=True)

    # Calculate inactive_hours column
    df['inactive_hours'] = df['inactive_since'].apply(
        lambda x: round((datetime.now() - x).total_seconds() / 3600, 2) if x != 'No Data' else x
    )

    # Calculate planned_end_time column
    df['planned_end_time'] = df['timeRemainingSecs'].apply(
        lambda x: datetime.now() + timedelta(seconds=x + 3) if x != 'No Data' else x
    )

    # Calculate stops_completed column
    df['stops_completed'] = np.where(df['actionedTimeWindowed'] == 0, df['completed'],
                                     df['total'] - df['notStarted'] - df['inProgress'])

    # Extract data from stopsAndPackagesByTaskAssessment column
    df['stops_at_risk'] = df['stopsAndPackagesByTaskAssessment'].apply(
        lambda x: x['AT_RISK']['stopsImpacted'] if x else 'No Data'
    )
    df['pkgs_at_risk'] = df['stopsAndPackagesByTaskAssessment'].apply(
        lambda x: x['AT_RISK']['packagesImpacted'] if x else 'No Data'
    )
    df['stops_ahead'] = df['stopsAndPackagesByTaskAssessment'].apply(
        lambda x: x['AHEAD']['stopsImpacted'] if x else 'No Data'
    )
    df['pkgs_ahead'] = df['stopsAndPackagesByTaskAssessment'].apply(
        lambda x: x['AHEAD']['packagesImpacted'] if x else 'No Data'
    )

    # Calculate shift_hours_remaining column
    df['shift_hours_remaining'] = df['timeRemainingSecs'].apply(
        lambda x: round(x / 3600, 2) if x != 'No Data' else x
    )
    
    
    # Define required columns
    req_cols = ['updated_timestamp', 'station_code', 'transporterId', 'driver_name', 'amazon_login_time',
                'planned_end_time', 'shift_hours_remaining', 'inactive_since', 'location_updated_time',
                'lastDriverEventTime', 'inactive_hours', 'executionStatus', 'driverSessionEnded',
                'stops_assigned', 'stops_completed', 'progressStatus', 'stops_at_risk', 'pkgs_at_risk',
                'stops_ahead', 'pkgs_ahead', 'totalPackages', 'UNASSIGNED', 'DELIVERED', 'REMAINING',
                'REATTEMPTABLE', 'UNDELIVERABLE', 'RETURNED']

    # Remove redundant columns
    req_cols.remove('location_updated_time')
    req_cols.remove('lastDriverEventTime')

    # Add missing columns with default value 0
    for col in req_cols:
        if col not in df.columns:
            df[col] = 0

    # Select the final columns
    final_df = df[req_cols]

    # Rename the columns
    final_df = final_df.rename(columns={'driverSessionEnded': 'driver_session_ended',
                                        'transporterId': 'transporter_id',
                                        'executionStatus': 'trip_status',
                                        'lastDriverEventTime': 'last_driver_event_time',
                                        'progressStatus': 'progress_status',
                                        'totalPackages': 'pkgs_assigned',
                                        'UNASSIGNED': 'pkgs_unassigned',
                                        'DELIVERED': 'pkgs_delivered',
                                        'REMAINING': 'pkgs_remaining',
                                        'REATTEMPTABLE': 'pkgs_reattemptable',
                                        'UNDELIVERABLE': 'pkgs_undeliverable',
                                        'RETURNED': 'pkgs_rts_done'})
    
    return final_df

    """
    This total function taking data frame as input, and cleaning it based on requirements.
    After that it return a final dataframe for future process.
    """

#### **Function 5 code** 

In [ ]:
def split_the_df(input_df):
    """
    Split the input DataFrame into multiple DataFrames based on specific conditions and return a dictionary
    containing the split DataFrames.

    Args:
        input_df (pd.DataFrame): Input DataFrame to be split.

    Returns:
        dict: A dictionary containing the split DataFrames.

    """
    df = input_df.copy()

    # Format date columns
    date_cols = ['amazon_login_time', 'inactive_since', 'planned_end_time']
    for col in date_cols:
        df[col] = df[col].apply(lambda x: x.strftime("%d/%m/%Y %H:%M") if x != 'No Data' else x)

    # Format progress status column
    df['progress_status'] = df['progress_status'].str.replace('_', ' ').str.title()

    # Split inactive drivers DataFrame
    inactive_df_cols = ['station_code', 'updated_timestamp', 'transporter_id', 'driver_name',
                        'progress_status', 'amazon_login_time', 'inactive_since', 'shift_hours_remaining',
                        'stops_assigned', 'stops_completed', 'stops_at_risk', 'pkgs_at_risk',
                        'om_name', 'rm_name', 'client']
    inactive_drivers = df[(df.inactive_hours >= 1) & (df.trip_status == 'DEPARTED')][inactive_df_cols].reset_index(drop=True)

    # Split not departed drivers DataFrame
    not_departed_df_cols = ['station_code', 'updated_timestamp', 'transporter_id', 'driver_name', 'amazon_login_time',
                            'stops_assigned', 'stops_completed', 'om_name', 'rm_name', 'client']
    not_departed_drivers = df[df.trip_status == 'NOT_DEPARTED'][not_departed_df_cols].reset_index(drop=True)

    # Split behinders DataFrame
    behinders_cols = ['station_code', 'updated_timestamp', 'transporter_id', 'driver_name', 'amazon_login_time',
                      'planned_end_time', 'shift_hours_remaining', 'inactive_hours', 'stops_assigned',
                      'stops_completed', 'stops_at_risk', 'om_name', 'rm_name', 'client']
    behinders = df[df.stops_at_risk != 'No Data']
    behinders = behinders[behinders.stops_at_risk >= 20][behinders_cols].reset_index(drop=True)

    # Split deliveries DataFrame
    deliveries_cols = ['station_code', 'updated_timestamp', 'transporter_id', 'driver_name', 'amazon_login_time',
                       'pkgs_delivered', 'pkgs_rts_done', 'pkgs_reattemptable', 'stops_assigned',
                       'stops_completed', 'om_name', 'rm_name', 'client']
    deliveries_df = df[deliveries_cols]
    deliveries_df = deliveries_df.replace('', np.nan).fillna(0)

    # Return the split DataFrames as a dictionary
    return {'inactive_drivers_df': inactive_drivers,
            'not_departed_drivers_df': not_departed_drivers,
            'behinders_df': behinders,
            'deliveries_df': deliveries_df}



    """
    This function is same as function 4 that take pandas data frame as input,
    manipulation data and return a Python dictionary for future process.
    """


#### **Function 6 code** 

In [ ]:
def get_current_drivers_data(sample):
    """
    Retrieve current drivers' data for specified sample size or all sites.

    Args:
        sample (bool): Flag indicating whether to use a sample size or retrieve data for all sites.

    Returns:
        dict: A dictionary containing the retrieved drivers' data in different DataFrames.

    Raises:
        custom_errors.NoDataError: If no data is retrieved for any site.

    """
    current_datetime = datetime.now(config.tz)
    date_str = current_datetime.strftime("%Y-%m-%d")
    
    
    
    amazon_sites_df_raw = amazon_sites.create_amazon_sites_df(
        db=db_connector.connect_to_db(db_name='whizzard'))
    amz_sites_fpath = f'/home/ubuntu/atom/{PROJECT_NAME}/amazon_sites.xlsx' \
        if config.ON_SERVER else f'../{PROJECT_NAME}/amazon_sites.xlsx'
    amazon_sites_df = toolkit.save_or_retrieve_df_excel(
        input_df=amazon_sites_df_raw, fpath=amz_sites_fpath)
    service_area_id_df = get_service_area_id()
    sites_service_id_df = pd.merge(toolkit.snake_case_the_cols(amazon_sites_df),
                                   service_area_id_df[[
                                       'site_code', 'service_area_id']],
                                   on='site_code', how='left')

    stn_code_id_list = sites_service_id_df[[
        'site_code', 'service_area_id']].to_numpy().tolist()
    print(f'Total Number of Sites found : {len(stn_code_id_list)}')
    stn_code_id_list = [row for row in stn_code_id_list if row[0] in [
        'HYDC', 'HYBH']] if sample else stn_code_id_list
    print(f'Sample Size (Number of Sites) : {len(stn_code_id_list)}') if sample else print(
        end='')
    
    
    
    drivers_df_list = []
    num = 0
    for stn, s_area_id in stn_code_id_list:
        num += 1
        try:
            drivers_df = get_drivers_data(
                yyyy_mm_dd=date_str, service_area_id=s_area_id)
            if not drivers_df.empty:
                drivers_df.insert(loc=0, column='date', value=date_str)
                drivers_df.insert(loc=1, column='updated_timestamp',
                                  value=current_datetime.strftime("%d/%m/%Y %H:00"))
                drivers_df.insert(loc=2, column='station_code', value=stn)
                drivers_df_list.append(drivers_df)
                print(f'{num}. {stn} - success', end=' ')
            else:
                print(f'{num}. {stn} - no_data', end=' ')
        except Exception as err:
            error_name = type(err).__name__
            print(f'{num}. {stn} - {error_name}', end=' ')
        if num % 8 == 0:
            print()
    print()

    if drivers_df_list:
        defaulters_df_raw = pd.concat(drivers_df_list)
    else:
        raise custom_errors.NoDataError

    defaulters_df = manipulate_the_data(input_df=defaulters_df_raw)

    site_details_df_raw = amazon_sites_df.drop(columns='Client Site Code')
    site_details_df = site_details_df_raw.rename(
        columns={'Site Code': 'station_code'})
    site_details_df = toolkit.snake_case_the_cols(input_df=site_details_df)
    defaulters_df_final = pd.merge(
        defaulters_df, site_details_df, on='station_code', how='left')
    splitted_dfs = split_the_df(input_df=defaulters_df_final)
    
    
    
    # splitted_dfs['inactive_drivers_df'].columns = ['Site', 'Updated Time', 'Driver ID', 'Name', 'Status',
    #                                                'Amz Login Time', 'Inactive From', 'Shift Hours Left',
    #                                                'Total Stops', 'Completed Stops', 'Stops at Risk',
    #                                                'Packages at Risk', 'OM', 'RM', 'Client']
    # splitted_dfs['not_departed_drivers_df'].columns = ['Site', 'Updated Time', 'Driver ID', 'Name', 'Amz Login Time',
    #                                                    'Total Stops', 'Completed Stops', 'OM', 'RM', 'Client']
    # splitted_dfs['behinders_df'].columns = ['Site', 'Updated Time', 'Driver ID', 'Name', 'Amz Login Time',
    #                                         'Shift Ends at', 'Shift Hours Left', 'Inactive Hours', 'Total Stops',
    #                                         'Completed Stops', 'Stops at Risk', 'OM', 'RM', 'Client']
    # splitted_dfs['deliveries_df'].columns = ['Site', 'Updated Time', 'Driver ID', 'Name', 'Amz Login Time',
    #                                          'Delivered', 'RTS Done', 'Reattemptable', 'Total Stops',
    #                                          'Completed Stops', 'OM', 'RM', 'Client']
    
    
    
    return {'defaulters_df_final': defaulters_df_final,
            'defaulters_df_raw': defaulters_df_raw,
            'inactive_drivers_df': splitted_dfs['inactive_drivers_df'],
            'not_departed_drivers_df': splitted_dfs['not_departed_drivers_df'],
            'behinders_df': splitted_dfs['behinders_df'],
            'deliveries_df': splitted_dfs['deliveries_df']}


    """
    This part is totaly work with private library
    """


#### **Script running code** ####

In [ ]:
if __name__ == '__main__':
    load_dotenv()
    data_folderpath = toolkit.create_folder(projectname=PROJECT_NAME, foldername='data')
    final_output_fname = f'{datetime.now(config.tz).strftime("%Y-%m-%d_AmazonDefaulters_%HHrs")}.xlsx'
    final_output_fpath = data_folderpath + '/' + final_output_fname
    t1 = time.time()  # execution start time for the python script

    google_creds_fpath = f'/home/ubuntu/atom/{PROJECT_NAME}/google_account_credentials.json' if config.ON_SERVER \
        else f'/Users/Admin/PycharmProjects/{PROJECT_NAME}/google_account_credentials.json'
    """
    This part of is setup the script running environment for server help of dotenv,
    and based on server business requirements.
    """
    
    
    
    try:
        print('\n--------------------***--------------------\n')
        print(f'Execution Started at: {datetime.now(config.tz).strftime("%Y-%m-%d %H:%M:%S")}')

        # Retrieve current drivers' data
        final_dfs = get_current_drivers_data(sample=SAMPLE)

        with pd.ExcelWriter(final_output_fpath, engine=None) as writer:
            final_dfs['inactive_drivers_df'].to_excel(writer, sheet_name='Inactive', index=False)
            final_dfs['not_departed_drivers_df'].to_excel(writer, sheet_name='Not Departed', index=False)
            final_dfs['behinders_df'].to_excel(writer, sheet_name='Behinders', index=False)
            final_dfs['deliveries_df'].to_excel(writer, sheet_name='Packages', index=False)
            
            """
            This part of code write excel files with data frames.
            """

            
            
        
        s3_foldername = 'whiz-amz-cortex-live'
        s3_storage = s3c.connect_to_s3_storage(os.getenv('AWS_ACCESS_KEY_ID'), os.getenv('AWS_SECRET_ACCESS_KEY'))
        atom_bucket = s3_storage.Bucket('atom-s3')  # Selecting a bucket from the S3 storage

        if not SAMPLE and UPLOAD_TO_S3:
            s3c.upload_to_s3(atom_bucket, s3_foldername, final_output_fpath, final_output_fname)

        # Get the latest 40 packages history data from S3
        req_excel_files = sorted(s3c.get_all_excels(connected_bucket=atom_bucket, folder_name=s3_foldername),
                                 reverse=True)[:40]
        history_pkgs_df = s3c.concat_excel_sheets_to_df(connected_bucket=atom_bucket,
                                                        excel_files=req_excel_files,
                                                        sheet_num='Packages')
        
        """
        This part of the code setup amazon S3 bucket to store data,
        and also ubload requirement data.
        This library documention is also private.
        """
        
        
        
        
        if UPDATE_GSHEET:
            gc.upload_df_to_gsheets(creds_fpath=google_creds_fpath,
                                    dataframe=final_dfs['inactive_drivers_df'],
                                    file_name="amazon-cortex-live-data",
                                    sheet_name="Inactive", formatting=False)
            gc.upload_df_to_gsheets(creds_fpath=google_creds_fpath,
                                    dataframe=final_dfs['not_departed_drivers_df'],
                                    file_name="amazon-cortex-live-data",
                                    sheet_name="Not Departed", formatting=False)
            gc.upload_df_to_gsheets(creds_fpath=google_creds_fpath,
                                    dataframe=final_dfs['behinders_df'],
                                    file_name="amazon-cortex-live-data",
                                    sheet_name="Behinders", formatting=False)
            gc.upload_df_to_gsheets(creds_fpath=google_creds_fpath,
                                    dataframe=final_dfs['deliveries_df'],
                                    file_name="amazon-cortex-live-data",
                                    sheet_name="Packages", formatting=False)
            gc.upload_df_to_gsheets(creds_fpath=google_creds_fpath,
                                    dataframe=history_pkgs_df,
                                    file_name="amazon-cortex-live-data",
                                    sheet_name="History - Packages", formatting=False)
            """
            This part is upload data frame to a goole sheet,
            the library documention is private.
            """
            
            
        
        
        print(f'Total Time for the Complete Execution: {(time.time() - t1) / 60:.3f} minutes')
        print(f'Execution Completed at: {datetime.now(config.tz).strftime("%Y-%m-%d %H:%M:%S")}')
        """
        This part of code printing total time of execution of the script,
        and also print execution server time.
        
        This part is for monitor the script.
        """
        
        
        
    except Exception as e:
        """
        This part of code handle error,
        if there is any error it will notify developer via emails for recheck. 
        """
        message = f'Error occurred while generating the report!\nError: {type(e).__name__}, {e}'
        print(message)
        subj = f'Failed: {final_output_fname.replace(".csv", "")}'
        toolkit.send_failure_email(send=SEND_FAIL_EMAIL, from_email=os.getenv('EMAIL_ID'),
                                   pwd=os.getenv('EMAIL_PASSWORD'),
                                   receiver_email='lafir.malim@whizzard.in',
                                   email_subject=subj, email_message=message)

In [10]:

import os
import json
import time
from datetime import datetime, timedelta
"""
This part of library are Python inbuild.
- os Library for setup path for server
- json Libray for converting API data to Python object
- datetime and 'timedelta' for converting date object to Python object and 'datetime' is for geting current datetime
"""


import numpy as np
import pandas as pd
import requests
from dotenv import load_dotenv
# noinspection SpellCheckingInspection  (Function 1)
def get_service_area_id():
    """
    Retrieves service area IDs from an API endpoint.

    Returns:
        pd.DataFrame: DataFrame containing service area information.
    """

    payload = ""
    headers = {"cookie": "session-id=260-6428466-7223316; session-id-time=2082787201l; "
                         "session-token=GbElPtRlx37WtbcOToTC3cRl2schxCQbGfw9PermyrbyTyonOdmMyDvN9xAn3Y1u"
                         "urDFHnNpykoR0H9l8R%2Bw0pqKOD14Zwtk4QClUVQpQsIItI6ETeBDN0M5SpFCUku0BGvpR2xSOFJN"
                         "nMUMDfYiflLntxjcLhKYtat1HekaJ2LZEwTlJ3dL6%2Fs%2B9U37Zu8JIQq%2FK7qPG5HFr4SwvqwIoQ"
                         "pyvIMoFI7ZbtIDa1gNSij8nXYZmqmACRQ0FUVbIFHYlcHjlwmkXzg%3D"}

    url = "https://logistics.amazon.in/flex/api/getOperationalRegions"
    response = requests.get(url, data=payload, headers=headers)

    data = json.loads(response.text)
    req_data = data
    api_df_raw = pd.DataFrame(req_data)
    
    """
    Payload, Header, and URl is sending request materials.
    Header conferm, we get write data based on our requirements and URL is parrent path.
    Request library under the hood make this into url raw string, that server can understand.
    
    We converted APIs json response to Python json object help of json.loads()
    We converted json object to pandas data frame 'api_df_raw'
    """
    
    

    # Explode the 'basicServiceAreas' column into multiple rows
    api_df = api_df_raw.explode('basicServiceAreas')

    # Split the exploded column into separate columns
    api_df_inter = pd.concat(
        [
            api_df.drop(['basicServiceAreas'], axis=1),
            api_df['basicServiceAreas'].apply(pd.Series)
        ],
        axis=1
    )

    # Concatenate the necessary columns for the final DataFrame
    df_final = pd.concat(
        [
            api_df_inter.drop(['pickUpLocationAddress', 'pickUpLocation'], axis=1),
            api_df_inter['pickUpLocationAddress'].apply(pd.Series),
            api_df_inter['pickUpLocation'].apply(pd.Series)
        ],
        axis=1
    )

    # Select the desired columns and rename them
    final_df = df_final[['defaultStationCode', 'serviceAreaID', 'regionID', 'regionName',
                         'state', 'postalCode', 'longitude', 'latitude', 'active']]
    final_df.columns = ['site_code', 'service_area_id', 'region_id', 'region_name',
                        'state', 'postal_code', 'longitude', 'latitude', 'active']
    
    """
    This part doing data manupulation task based on business requirements, and return a final clean data frame
    """
    

    return final_df

df = get_service_area_id()
df.head()

,site_code,service_area_id,region_id,region_name,state,postal_code,longitude,latitude,active
0,DAK1,ba419f3f-d223-44d4-83fc-a87212a83fbe,19920c05-6bb1-429f-b995-444e3b8bbb61,Abha,Abha,20642,42.649750,18.254000,True
1,AGRD,b8e4652b-cea4-4e1b-b61f-902a4e687b21,39f2ef6a-4294-4066-b5d7-86dc9ac811e5,Agra,UP,282006,78.062374,27.212077,True
1,FUPD,8e97362b-0951-406d-b2ea-9aa1fef9660e,39f2ef6a-4294-4066-b5d7-86dc9ac811e5,Agra,UTTAR PRADESH,282001,82.923594,25.286959,True
1,PNAM,ac3aa2bc-98d2-4ce6-9b36-c8b88161ba95,39f2ef6a-4294-4066-b5d7-86dc9ac811e5,Agra,UTTAR PRADESH,282007,78.053900,27.148700,True
1,UPAA,49c0a838-e0ea-4899-84d5-829a9eb87bdb,39f2ef6a-4294-4066-b5d7-86dc9ac811e5,Agra,Uttar Pradesh,282007,0.000000,0.000000,True
